In [88]:
import numpy as np
import pandas as pd
import itertools
import typing
from enum import Enum, unique

In [89]:
@unique
class Artificial(IntEnum):
    ORIGENES = 1
    DESTINOS = 2

In [2]:
oferta_origenes = []
demanda_destinos = []
num_origenes = 0
num_destinos = 0
num_transbordos = 0
mat_costo = []
mat_sombra = []
mat_dec = []
mat_base = []
artificial = 0

In [3]:
def input_int(message):
    return int(input(message))

def try_again():
    input_int("")

In [4]:
def inicializarVariables():
    global oferta_origenes
    global demanda_destinos
    global num_origenes
    global num_destinos
    global num_transbordos
    global mat_costo
    global mat_sombra
    global mat_dec
    global mat_base
    global artificial
    oferta_origenes = []
    demanda_destinos = []
    num_origenes = 0
    num_destinos = 0
    num_transbordos = 0
    mat_costo = []
    mat_sombra = []
    mat_dec = []
    mat_base = []
    artificial = 0

In [90]:
# Solicitar Variables
def solicitarDatos():
    """Esta función es para llenar los datos usados en el modelo.
    Se solicita mediante input en consola.
    Pedimos 3 cosas:
     * Pedimos la cantidad de origenes, destinos y transbordos.
     * Pedimos las ofertas de cada origen y demandas de cada destino.
     * Pedimos el costo de de cada posible transicion.
    """
    print("Vamos a resolver un problemas de transporte.")
    try:
        global oferta_origenes
        global demanda_destinos
        global num_origenes
        global num_destinos
        global num_transbordos
        global mat_costo
        global artificial
        
        # Cantidad de orignes, destinos y transbordos. 
        num_origenes = input_int("¿Cuantos origenes tiene tu problema?")
        num_destinos = input_int("¿Cuantos destinos tiene tu problema?")
        num_transbordos = input_int("¿Cuantos transbordos?\n(Si no tiene transbordo pon 0)")
        
        # Ofertas de cada origen y demandas de cada destino
        for i in range(num_origenes):
            oferta_origenes.append(input_int("¿Cuál es la oferta de tu origen #{}".format(i+1)))
            
        oferta_transbordos = [sum(oferta_origenes)]*num_transbordos
        oferta_origenes += oferta_transbordos

        demanda_destinos += oferta_transbordos
        for i in range(num_destinos):
            demanda_destinos.append(input_int("¿Cuál es la demanda de tu destino #{}".format(i+1)))
        
        print(oferta_origenes, demanda_destinos)
            
        # Matriz de costos
        print("Excelente ahora llenemos la matriz de costos.")
        print("Si una conexion no es valida inserta una x.")
        for i in range(num_destinos+num_transbordos):
            mat_costo.append([])
        num_cols = num_destinos+num_transbordos
        num_filas = num_origenes+num_transbordos
        max_cant_conexiones = (num_filas)*(num_cols)
        
        for i in range(max_cant_conexiones):
            fila = i//num_filas # ORIGENES
            col = i%num_cols # DESTINOS
            origen = "origen" if fila<num_origenes else "transbordo"
            n_origen = fila+1 if fila<num_origenes else fila+1-num_origenes
            destino = "transbordo" if col<num_transbordos else "destino"
            n_destino = col+1 if col<num_transbordos else col+1-num_transbordos
            temp_val = input("Costo de {} #{} al {} #{} es:".format(origen, n_origen, destino, n_destino))
            # TODO: Validaciones de la entrada de costos
            temp_val = int(temp_val) if temp_val!="x" or temp_val1=="X" else -1
            temp_val = temp_val if temp_val>=0 else -1
            
            mat_costo[i//num_filas].append(temp_val
                                          )
        if sum(oferta_origenes)>sum(demanda_destinos):
            artificial = Artificial.DESTINOS
            demanda_destinos.append(sum(oferta_origenes)-sum(demanda_destinos))
            for i in range(num_filas):
                mat_costo[i].append(0)
                
        elif sum(oferta_origenes)<sum(demanda_destinos):
            artificial = Artificial.ORIGENES
            oferta_origenes.append(sum(demanda_destinos)-sum(oferta_origenes))
            mat_costo.append([0]*num_cols)
               
        oferta_origenes = np.array(oferta_origenes)
        demanda_destinos = np.array(demanda_destinos)
        mat_costo = np.array(mat_costo)
        
    except ValueError:
        print("No entendí eso. :( Porfavor solo inserta valores numéricos")
        print("Enter para intentar otra vez.")
        inicializarVariables()
        input()
        solicitarDatos()
    except BaseException as e:
        print("Ups parece que hubo un error!")
        print(e)
        e.traceback()
        print("Enter para intentar otra vez.")
        inicializarVariables()
        input()
        solicitarDatos()


In [91]:
# inicializarVariables()
# solicitarDatos()

num_origenes = 2
num_destinos = 2
num_transbordos = 2
oferta_origenes = np.array([150,200,350,350])
demanda_destinos = np.array([350,350,130,130,90])
artificial = Artificial.DESTINOS

mat_costo = np.array([
    [8, 13, 25, 28, 0],
    [15, 12, 26, 25, 0],
    [0, 6, 16, 17, 0 ],
    [6, 0, 14, 16, 0]
])

In [7]:
def noroeste(oOfertas: np.ndarray, oDemandas: np.ndarray, oTransbordos: int = 0) -> np.ndarray:
    ofertas = oOfertas.copy()
    demandas = oDemandas.copy()
    transbordos = oTransbordos
    print(ofertas, demandas, transbordos)
    sobrante_oferta = ofertas.sum()
    origen_idx = 0
    destino_idx = 0
    
    mat_dec = np.zeros((len(ofertas), len(demandas)))
    mat_base = np.full((len(ofertas), len(demandas)), False)
    
    while sobrante_oferta > 0:
        current = min(ofertas[origen_idx], demandas[destino_idx])
        mat_dec[origen_idx][destino_idx] = current
        mat_base[origen_idx][destino_idx] = True
        sobrante_oferta -= current
        ofertas[origen_idx] -= current
        demandas[destino_idx] -= current
        if ofertas[origen_idx] <= demandas[destino_idx]:
            origen_idx += 1
        else:
            destino_idx +=1
    return mat_dec, mat_base
    

In [8]:
mat_dec, mat_base = noroeste(oferta_origenes, demanda_destinos, num_transbordos)
mat_dec, mat_base

[150 200 350 350] [350 350 130 130  90] 2


(array([[150.,   0.,   0.,   0.,   0.],
        [200.,   0.,   0.,   0.,   0.],
        [  0., 350.,   0.,   0.,   0.],
        [  0.,   0., 130., 130.,  90.]]),
 array([[ True, False, False, False, False],
        [ True, False, False, False, False],
        [ True,  True, False, False, False],
        [False,  True,  True,  True,  True]]))

In [121]:
def costo_minimo(oOfertas: np.ndarray , oDemandas: np.ndarray , oCostos: np.ndarray, oTransbordos: int = 0) -> np.ndarray:
    ofertas = oOfertas.copy()
    demandas = oDemandas.copy()
    costos = oCostos.copy()
    transbordos = oTransbordos
    print("ofertas", ofertas)
    print("demandas", demandas)
    print("costos", costos)
    print("trasnbordos", transbordos)
    
    sobrante_oferta = ofertas.sum()
    sobrante_origenes = ofertas[0:len(ofertas)-transbordos].sum()
    sobrante_demandas = demandas[transbordos:-1].sum() if artificial == Artificial.DESTINOS else demandas[transbordos:].sum()
    origenes_disponibles = [True]*len(ofertas)
    destinos_disponibles = [True]*len(demandas)
    
    mat_dec = np.zeros((len(ofertas), len(demandas)))
    mat_base = np.full((len(ofertas), len(demandas)), False)
    
    counter = 0
    
    import pdb
    
    while sobrante_origenes > 0 and counter < 20:
        costos_origenes = costos[0:len(ofertas)-transbordos,]
        costos_origenes = costos_origenes[origenes_disponibles[0:len(ofertas)-transbordos]]
        costos_origenes = costos_origenes[:,destinos_disponibles]
        minimo = costos_origenes.min()
        minimo = minimo if minimo > 0 else np.partition(np.unique(costos_origenes), 1)[1]
        fila = np.where(costos_origenes == minimo)[0]
        fila = fila[0]
        col = np.where(costos_origenes[fila] == minimo)[0][0]
        
        current = min(ofertas[origenes_disponibles][fila], demandas[destinos_disponibles][col])
        
        campos_disponibles = [x[0] and x[1] for x in itertools.product(origenes_disponibles,destinos_disponibles)]
        campos_disponibles = np.array(campos_disponibles).reshape(len(ofertas), len(demandas))
        fila = np.where(np.logical_and(costos == minimo, campos_disponibles))[0][0]
        col = np.where(costos[fila] == minimo)[0][0]
        mat_dec[fila][col] = current
        mat_base[fila][col] = True
        
        sobrante_oferta -= current
        sobrante_origenes -= current
        ofertas[fila] -= current
        demandas[col] -= current
        if ofertas[fila] <= demandas[col]:
            origenes_disponibles[fila] = False
        else:
            destinos_disponibles[col] = False
        counter += 1
        
    counter = 0

            
    while sobrante_demandas > 0 and counter < 20:
        costos_demandas = costos[:,transbordos:]
        costos_demandas = costos_demandas[origenes_disponibles,:]
        costos_demandas = costos_demandas[:,destinos_disponibles[transbordos:]]
        
        minimo = costos_demandas.min()
        minimo = minimo if minimo > 0 else np.partition(np.unique(costos_demandas), 1)[1]
        fila = np.where(costos_demandas == minimo)[0]
        fila = fila[0]
        col = np.where(costos_demandas[fila] == minimo)[0][0]
        
        current = min(ofertas[origenes_disponibles][fila], demandas[destinos_disponibles][transbordos:][col])
        
        campos_disponibles = [x[0] and x[1] for x in itertools.product(origenes_disponibles,destinos_disponibles)]
        campos_disponibles = np.array(campos_disponibles).reshape(len(ofertas), len(demandas))
        fila = np.where(np.logical_and(costos == minimo, campos_disponibles))[0][0]
        col = np.where(costos[fila] == minimo)[0][0]
        mat_dec[fila][col] = current
        mat_base[fila][col] = True
        
        sobrante_oferta -= current
        sobrante_demandas -= current
        ofertas[fila] -= current
        demandas[col] -= current
        if ofertas[fila] <= demandas[col]:
            origenes_disponibles[fila] = False
        else:
            destinos_disponibles[col] = False
        counter += 1

    counter = 0

            
    while sobrante_oferta > 0 and counter < 20:
        pdb.set_trace()
        costos_sobrantes = costos[origenes_disponibles,:]
        costos_sobrantes = costos_sobrantes[:,destinos_disponibles]
        
        minimo = costos_sobrantes.min()
#         minimo = minimo if minimo > 0 else np.partition(np.unique(costos_sobrantes), 1)[1]
        fila = np.where(costos_sobrantes == minimo)[0]
        fila = fila[0]
        col = np.where(costos_sobrantes[fila] == minimo)[0][0]
        
        current = min(ofertas[origenes_disponibles][fila], demandas[destinos_disponibles][col])
        
        campos_disponibles = [x[0] and x[1] for x in itertools.product(origenes_disponibles,destinos_disponibles)]
        campos_disponibles = np.array(campos_disponibles).reshape(len(ofertas), len(demandas))
        fila = np.where(np.logical_and(costos == minimo, campos_disponibles))[0][0]
        col = np.where(np.logical_and(costos == minimo, campos_disponibles))[1][0]
        mat_dec[fila][col] = current
        mat_base[fila][col] = True
        
        sobrante_oferta -= current
        sobrante_demandas -= current
        ofertas[fila] -= current
        demandas[col] -= current
        if ofertas[fila] <= demandas[col]:
            origenes_disponibles[fila] = False
        else:
            destinos_disponibles[col] = False
        print(mat_dec)
        print(mat_base)
        print("origenes disponibles: ",origenes_disponibles)
        print("destinos disponibles: ",destinos_disponibles)
            
    return mat_dec, mat_base

In [122]:
mat_dec, mat_base = costo_minimo(oferta_origenes, demanda_destinos, mat_costo, num_transbordos)
mat_dec, mat_base

ofertas [150 200 350 350]
demandas [350 350 130 130  90]
costos [[ 8 13 25 28  0]
 [15 12 26 25  0]
 [ 0  6 16 17  0]
 [ 6  0 14 16  0]]
trasnbordos 2
> <ipython-input-121-d90ba9d6e13f>(91)costo_minimo()
-> costos_sobrantes = costos[origenes_disponibles,:]


(Pdb)  n


> <ipython-input-121-d90ba9d6e13f>(92)costo_minimo()
-> costos_sobrantes = costos_sobrantes[:,destinos_disponibles]


(Pdb)  n


> <ipython-input-121-d90ba9d6e13f>(94)costo_minimo()
-> minimo = costos_sobrantes.min()


...

(Pdb)  c


[[150.   0.   0.   0.   0.]
 [  0. 200.   0.   0.   0.]
 [200.   0.   0.   0.  90.]
 [  0.  90. 130. 130.   0.]]
[[ True False False False False]
 [False  True False False False]
 [ True False False False  True]
 [False  True  True  True False]]
origenes disponibles:  [False, False, True, False]
destinos disponibles:  [False, True, False, False, False]
> <ipython-input-121-d90ba9d6e13f>(91)costo_minimo()
-> costos_sobrantes = costos[origenes_disponibles,:]


(Pdb)  c


[[150.   0.   0.   0.   0.]
 [  0. 200.   0.   0.   0.]
 [200.  60.   0.   0.  90.]
 [  0.  90. 130. 130.   0.]]
[[ True False False False False]
 [False  True False False False]
 [ True  True False False  True]
 [False  True  True  True False]]
origenes disponibles:  [False, False, False, False]
destinos disponibles:  [False, True, False, False, False]


(array([[150.,   0.,   0.,   0.,   0.],
        [  0., 200.,   0.,   0.,   0.],
        [200.,  60.,   0.,   0.,  90.],
        [  0.,  90., 130., 130.,   0.]]),
 array([[ True, False, False, False, False],
        [False,  True, False, False, False],
        [ True,  True, False, False,  True],
        [False,  True,  True,  True, False]]))

In [123]:
def preciosSombra():
    pass

In [ ]:
num_origenes, num_destinos, num_transbordos, oferta_origenes, demanda_destinos

In [ ]:
mat_costo, mat_dec, mat_sombra

In [ ]:
mat_costo[0], mat_costo

In [ ]:
test = np.array(mat_costo)

In [ ]:
test[0][0]

In [ ]:
test[0][0]=1

In [17]:
np.logical_and([True, False, True],[True,True,False])

array([ True, False, False])

In [20]:
a = np.matrix([True, False, True])
a.T

matrix([[ True],
        [False],
        [ True]])

[True, True, False, False, False, False, True, True, False]